In [108]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
# to use variable from other notebook
import import_ipynb
from covid_policy_cases_deaths import df_deaths, df_cases, series_to_supervised,\
    prep_ts, rename_cols_dict, state_dict, state_population, ts_death_df, states_in_policy,\
    #, ts_cases_df, df_policy

In [58]:
deaths_model = load_model('covid_deaths_model.h5')
cases_mode = load_model('covid_cases_model.h5')

In [66]:
df_deaths

,Date,State,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,Deaths Per Capita
38,2020-02-29,0,0.0000,0.0000,0.0000,0.00,0.000000
39,2020-03-01,0,0.0000,0.0000,0.0000,0.00,0.000000
40,2020-03-02,0,0.0000,0.0000,0.0000,0.00,0.000000
41,2020-03-03,0,0.0000,0.0000,0.0000,0.00,0.000000
42,2020-03-04,0,0.0000,0.0000,0.0000,0.00,0.000000
...,...,...,...,...,...,...,...
14503,2020-10-22,51,0.4074,0.4524,0.4861,0.25,0.000117
14504,2020-10-23,51,0.4074,0.4524,0.4861,0.25,0.000117
14505,2020-10-24,51,0.4074,0.4524,0.4861,0.25,0.000117
14506,2020-10-25,51,0.4074,0.4524,0.4861,0.25,0.000117


In [48]:
org_df_policy = pd.read_csv ('usa-covid-policy/data/OxCGRT_US_latest.csv')
org_jhu_deaths_df = pd.read_csv('csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
org_jhu_cases_df = pd.read_csv('csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

/home/nick/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [79]:
# Function to prepare the oxford policy data - like df_policy from covid_policy_cases_deaths file

def prep_oxford_data(oxford_data):
    # Remove irrelevant columns
    df_policy = oxford_data[["Date", "RegionName", "CountryCode", "Jurisdiction", "StringencyIndexForDisplay",
                           "GovernmentResponseIndexForDisplay", "ContainmentHealthIndexForDisplay",
                           "EconomicSupportIndexForDisplay"]]
    # Remove rows where the jursitiction is NAT_GOV then drop the jursidiction column
    df_policy.drop(df_policy[df_policy["Jurisdiction"] == "NAT_GOV"].index, inplace = True)
    df_policy = df_policy.drop(['Jurisdiction'], axis=1)
    # Normalize the policy index columns
    indices = df_policy.filter(regex="Index")
    for col in indices.columns:
        df_policy[col] /= 100
    # Remove old index column and reset it
    df_policy = df_policy.reset_index().drop("index", axis=1)
    # Make the date column datetime
    df_policy['Date'] = pd.to_datetime(df_policy['Date'], format='%Y%m%d')
    # This replaces all Nan Values where the previous value == the next valid value 
    # For the specified column - the Nan is replaced with the prev value
    df_policy.loc[df_policy['StringencyIndexForDisplay'].ffill() == 
                  df_policy['StringencyIndexForDisplay'].bfill(),
                  'StringencyIndexForDisplay'] = df_policy['StringencyIndexForDisplay'].ffill()

    df_policy.loc[df_policy['GovernmentResponseIndexForDisplay'].ffill() == 
                  df_policy['GovernmentResponseIndexForDisplay'].bfill(),
                  'GovernmentResponseIndexForDisplay'] = df_policy['GovernmentResponseIndexForDisplay'].ffill()

    df_policy.loc[df_policy['ContainmentHealthIndexForDisplay'].ffill() == 
                  df_policy['ContainmentHealthIndexForDisplay'].bfill(),
                  'ContainmentHealthIndexForDisplay'] = df_policy['ContainmentHealthIndexForDisplay'].ffill()

    df_policy.loc[df_policy['EconomicSupportIndexForDisplay'].ffill() == 
                  df_policy['EconomicSupportIndexForDisplay'].bfill(),
                  'EconomicSupportIndexForDisplay'] = df_policy['EconomicSupportIndexForDisplay'].ffill()
    # Checking there are no nulls in df_policy
    null_policy = df_policy[df_policy.isnull().any(axis=1)]
    if len(null_policy) != 0:
        print("WARNING: There are null values in your data - further cleaning required")
    return df_policy

In [ ]:
# z = prep_oxford_data(org_df_policy)
# z

In [133]:
# Note: states_in_policy was loaded in the import statement

def prep_jhu_deaths_data(df_deaths_jhu):
    # Group the data by state and reset index column
    df_deaths_jhu = org_jhu_deaths_df.groupby('Province_State').sum().reset_index()
    # Rename Washington DC so it matches the policy df and remove the other territories not in the policy df
    df_deaths_jhu['Province_State'].replace({"District of Columbia": "Washington DC"}, inplace=True)
    # Find and remove states / territories that are not in the policy df
    states_in_df_deaths = set(df_deaths_jhu['Province_State'])
    diff_states = states_in_df_deaths.difference(states_in_policy)
    df_deaths_jhu = df_deaths_jhu[~df_deaths_jhu['Province_State'].isin(diff_states)]
    # Remove all columns except the Province_State and the Dates
    df_deaths_jhu = df_deaths_jhu.drop(['UID', 'code3', 'FIPS', 'Lat', 'Long_', 'Population'], axis=1)
    # Create a list of the dates in the deaths df
    date_list = list(df_deaths_jhu.columns)[1:]
    # Copy your df and create a blank one
    temp_deaths_df = df_deaths_jhu.copy()
    deaths_df = pd.DataFrame(columns=('Date', 'State', 'Deaths Per Capita'))
    # fill in the blank df - this changes the dates from columns to rows      
    for index, row in temp_deaths_df.iterrows():
        for date in date_list:
            state = row['Province_State']
            new_row = pd.Series({'Date':date, 'State':state,
                                 'Deaths Per Capita':row[date] / 
                                 int(state_population[state_population['Province_State'] == state]['Population'])})
            deaths_df = deaths_df.append(new_row, ignore_index= True)
    # make date column datetime to match policy df
    deaths_df['Date'] = pd.to_datetime(deaths_df['Date'], format='%m/%d/%y')
    return deaths_df


In [134]:
# Note: states_in_policy was loaded in the import statement

def prep_jhu_cases_data(df_cases_jhu):
    # Group the data by state and reset index column
    df_cases_jhu = org_jhu_cases_df.groupby('Province_State').sum().reset_index()
    # Rename Washington DC so it matches the policy df and remove the other territories not in the policy df
    df_cases_jhu['Province_State'].replace({"District of Columbia": "Washington DC"}, inplace=True)
    # Find and remove states / territories that are not in the policy df
    states_in_df_cases = set(df_cases_jhu['Province_State'])
    diff_states = states_in_df_cases.difference(states_in_policy)
    df_cases_jhu = df_cases_jhu[~df_cases_jhu['Province_State'].isin(diff_states)]
    # Remove all columns except the Province_State and the Dates
    df_cases_jhu = df_cases_jhu.drop(['UID', 'code3', 'FIPS', 'Lat', 'Long_'], axis=1)
    # Create a list of the dates in the cases df
    date_list = list(df_cases_jhu.columns)[1:]
    # Copy your df and create a blank one
    temp_cases_df = df_cases_jhu.copy()
    cases_df = pd.DataFrame(columns=('Date', 'State', 'Cases Per Capita'))
    # fill in the blank df - this changes the dates from columns to rows   
    for index, row in temp_cases_df.iterrows():
        for date in date_list:
            state = row['Province_State']
            new_row = pd.Series({'Date':date, 'State':state,
                                 'Cases Per Capita': row[date] /
                                  int(state_population[state_population['Province_State'] == state]['Population'])})
            cases_df = cases_df.append(new_row, ignore_index= True)
    # make date column datetime to match policy df
    cases_df['Date'] = pd.to_datetime(cases_df['Date'], format='%m/%d/%y')
    return cases_df
    

In [167]:
# Note: the state_dict was loaded in the import statements

def merge_jhu_policy(start_date, end_date, jhu_df, df_policy):
    per_capita_col = jhu_df.columns[-1]
    jhu_df = jhu_df[jhu_df['Date'] >= start_date]
    jhu_df = jhu_df[jhu_df['Date'] <= end_date]
    df_policy = df_policy[df_policy['Date'] >= start_date]
    df_policy = df_policy[df_policy['Date'] <= end_date]
    df = pd.merge(jhu_df, df_policy, how='inner', left_on=['Date', 'State'], right_on=['Date', 'RegionName'])
    df["State"].replace(state_dict, inplace=True)
    cols = list(df.columns)
    cols.remove(per_capita_col)
    cols.append(per_capita_col)
    df = df[cols]
    df = df.drop(['RegionName', 'CountryCode'], axis=1)
    return df

Test below

In [160]:
c = prep_jhu_cases_data(org_jhu_cases_df)

In [161]:
d = prep_jhu_deaths_data(org_jhu_deaths_df)

In [162]:
p = prep_oxford_data(org_df_policy)

/home/nick/.local/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [168]:
combo = merge_jhu_policy('09/01/2020', '10/15/2020',c,p)


In [169]:
combo

,Date,State,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,Cases Per Capita
0,2020-09-01,0,0.3148,0.4524,0.4653,0.375,0.026027
1,2020-09-02,0,0.3148,0.4524,0.4653,0.375,0.026154
2,2020-09-03,0,0.3148,0.4524,0.4653,0.375,0.026368
3,2020-09-04,0,0.3148,0.4524,0.4653,0.375,0.026594
4,2020-09-05,0,0.3148,0.4524,0.4653,0.375,0.026881
...,...,...,...,...,...,...,...
2335,2020-10-11,51,0.4074,0.4524,0.4861,0.250,0.013151
2336,2020-10-12,51,0.4074,0.4524,0.4861,0.250,0.013481
2337,2020-10-13,51,0.4074,0.4524,0.4861,0.250,0.013760
2338,2020-10-14,51,0.4074,0.4524,0.4861,0.250,0.014129
